In [1]:
import matplotlib.pyplot as plt
import numpy as np
from custom_sepsis import *

In [2]:
ts = [{
    Simplification.SIMPLE: DirThompsonSampling(create_model(Simplification.SIMPLE), {},{}, {}, f"Simple-{i}", {}),
    Simplification.MEDIUM: DirThompsonSampling(create_model(Simplification.MEDIUM), {},{}, {}, f"Medium-{i}", {}),
    Simplification.NONE: FullThompsonSampling(create_model(Simplification.NONE), {},{}, {}, f"None-{i}", {}),
} for i in range(3)]
policies = [{
    Simplification.SIMPLE: random_policy(),
    Simplification.MEDIUM: random_policy(),
    Simplification.NONE: random_policy(),
} for i in range(3)]
rewards = [{
    Simplification.SIMPLE: {},
    Simplification.MEDIUM: {},
    Simplification.NONE: {},
} for i in range(3)]
model_types = [Simplification.SIMPLE, Simplification.MEDIUM, Simplification.NONE]

In [6]:
ts[0][Simplification.SIMPLE] = DirThompsonSampling.load("data/dir_ts/Simple-0.pkl.gz")
ts[1][Simplification.SIMPLE] = DirThompsonSampling.load("data/dir_ts/Simple-1.pkl.gz")
ts[2][Simplification.SIMPLE] = DirThompsonSampling.load("data/dir_ts/Simple-2.pkl.gz")
ts[0][Simplification.MEDIUM] = DirThompsonSampling.load("data/dir_ts/Medium-0.pkl.gz")
ts[1][Simplification.MEDIUM] = DirThompsonSampling.load("data/dir_ts/Medium-1.pkl.gz")
ts[2][Simplification.MEDIUM] = DirThompsonSampling.load("data/dir_ts/Medium-2.pkl.gz")
ts[0][Simplification.NONE] = DirThompsonSampling.load("data/dir_ts/None-0.pkl.gz")
ts[1][Simplification.NONE] = DirThompsonSampling.load("data/dir_ts/None-1.pkl.gz")
ts[2][Simplification.NONE] = DirThompsonSampling.load("data/dir_ts/None-2.pkl.gz")

In [7]:
start = 10000
nr_episodes = 1000000


In [ ]:
V = np.zeros(n_states)
for ep_ix in range(start+1, nr_episodes+1):
    for i in range(3):
        if (np.log2(ep_ix) % 1 == 0 and ep_ix<100) or (ep_ix % 100 == 0): # every power of 2 or 100 episodes, a new policy is computed
            for model_type in model_types:
                state_counts = ts[i][model_type].model.get_state_counts()
                policy, V = matrix_value_iteration(V, ts[i][model_type].model.transition_model())
                ts[i][model_type].add_data(ep_ix, rewards[i][model_type], policy, state_counts)
                ts[i][model_type].save()
                print(f"Episode {ep_ix}, {model_type}, {i} done")
                
        for model_type in model_types:
            episode = run_episode(policies[i][model_type])
            rewards[i][model_type][ep_ix] = np.sum(episode.rewards)
            ts[i][model_type].model.update_state_counts(episode)
            